In [2]:
%pip install mne_connectivity

   ---------------------------------------- 0.0/107.2 kB ? eta -:--:--
   -------------------------------------- - 102.4/107.2 kB 3.0 MB/s eta 0:00:01
   ---------------------------------------- 107.2/107.2 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/6.6 MB ? eta -:--:--
   - -------------------------------------- 0.3/6.6 MB 9.2 MB/s eta 0:00:01
   ---- ----------------------------------- 0.8/6.6 MB 8.4 MB/s eta 0:00:01
   ------- -------------------------------- 1.2/6.6 MB 8.5 MB/s eta 0:00:01
   --------- ------------------------------ 1.6/6.6 MB 8.5 MB/s eta 0:00:01
   ------------ --------------------------- 2.0/6.6 MB 8.5 MB/s eta 0:00:01
   -------------- ------------------------- 2.4/6.6 MB 8.5 MB/s eta 0:00:01
   ---------------- ----------------------- 2.8/6.6 MB 8.5 MB/s eta 0:00:01
   ------------------- -------------------- 3.2/6.6 MB 8.5 MB/s eta 0:00:01
   ------------------- -------------------- 3.3/6.6 MB 8.7 MB/s eta 0:00:01
   -------------

In [3]:
%matplotlib inline

import mne_connectivity
import numpy as np

from _helper_functions import simulate_connectivity, plot_connectivity

## Multivariate frequency-resolved connectivity - the `mne-connectivity` package continued

The connectivity methods covered so far have all been bivariate methods, i.e. connectivity from one signal to another signal.

In contrast, multivariate connectivity methods can be used to compute connectivity between whole groups of signals simultaneously, bringing both practical and methodological benefits.

### Part 1 - Simulating connectivity

As before, we will use the custom helper function `simulate_connectivity()` to generate signals which we can explore multivariate connectivity computations on.

**Exercises - Simulating connectivity**

**Exercise:** Simulate 2 interacting channels in the frequency ranges: 5-10 Hz; 15-20 Hz; and 25-30 Hz.

Do this in 3 separate function calls.

In [ ]:
##  CODE GOES HERE
epochs_5_10 = simulate_connectivity(n_seeds=1, n_targets=1, freq_band=(5, 10))
epochs_15_20 = simulate_connectivity(n_seeds=1, n_targets=1, freq_band=(15, 20))
epochs_25_30 = simulate_connectivity(n_seeds=1, n_targets=1, freq_band=(25, 30))

**Exercise:** Combine the 3 sets of `Epochs` into a single `Epochs` object using the [`add_channels()`](https://mne.tools/stable/generated/mne.Epochs.html#mne.Epochs.add_channels) method.

In [ ]:
## CODE GOES HERE
epochs = epochs_5_10.copy().add_channels([epochs_15_20, epochs_25_30])

**Exercise:** Verify that activity is present in the appropriate frequency ranges by computing the power spectra of the data.

In [ ]:
## CODE GOES HERE
epochs.compute_psd().plot();

### Part 2 - A recap of bivariate connectivity

Again, we will use [`mne_connectivity.spectral_connectivity_epochs()`](https://mne.tools/mne-connectivity/stable/generated/mne_connectivity.spectral_connectivity_epochs.html#mne_connectivity.spectral_connectivity_epochs) to compute connectivity between the simulated signals.

We first generate the results from a bivariate connectivity method to use as a comparison for the multivariate methods.

**Exercises - Bivariate connectivity**

**Exercise:** Compute connectivity using the imaginary part of coherency (`imcoh` method).

Specify the indices such that connectivity is only computed between the 3 sets of interacting channels.

In [4]:
 mne_connectivity.spectral_connectivity_epochs?

Signature:
mne_connectivity.spectral_connectivity_epochs(
    data,
    names=None,
    method='coh',
    indices=None,
    sfreq=None,
    mode='multitaper',
    fmin=None,
    fmax=inf,
    fskip=0,
    faverage=False,
    tmin=None,
    tmax=None,
    mt_bandwidth=None,
    mt_adaptive=False,
    mt_low_bias=True,
    cwt_freqs=None,
    cwt_n_cycles=7,
    gc_n_lags=40,
    rank=None,
    block_size=1000,
    n_jobs=1,
    verbose=None,
)
Docstring:
Compute frequency- and time-frequency-domain connectivity measures.

The connectivity method(s) are specified using the "method" parameter.
All methods are based on estimates of the cross- and power spectral
densities (CSD/PSD) Sxy and Sxx, Syy.

Parameters
----------
data : array-like, shape=(n_epochs, n_signals, n_times) | Epochs
    The data from which to compute connectivity. Note that it is also
    possible to combine multiple signals by providing a list of tuples,
    e.g., data = [(arr_0, stc_0), (arr_1, stc_1), (arr_2, stc_2)],

In [ ]:
## CODE GOES HERE
bivariate_connectivity = mne_connectivity.spectral_connectivity_epochs(
    data=epochs, method="imcoh", indices=([0, 2, 4], [1, 3, 5])
)

**Exercise:** Plot the connectivity results for each connection to verify the interaction is present.

*Hint:* Results for the imaginary part of coherency can be positive and negative. For our purposes, you should take the absolute values of the results using NumPy's [`abs()`](https://numpy.org/doc/stable/reference/generated/numpy.absolute.html) function.

In [ ]:
## CODE GOES HERE
for con_idx in range(len(bivariate_connectivity.indices[0])):
    plot_connectivity(
        np.abs(bivariate_connectivity.get_data()[con_idx]), bivariate_connectivity.freqs
    )

**Exercise:** Summarise the bivariate connectivity results by averaging across the 3 connections.

*Hint:* Use NumPy's [`mean()`](https://numpy.org/doc/stable/reference/generated/numpy.mean.html) function, and don't forget to take the absolute values first!

In [ ]:
## CODE GOES HERE
average_connectivity = np.mean(np.abs(bivariate_connectivity.get_data()), axis=0)

**Exercise:** Plot the average bivariate connectivity results using the custom `plot_connectivity()` helper function as before.

What do you notice about the scale of the values? What is the reason for this?

In [ ]:
## CODE GOES HERE
plot_connectivity(average_connectivity, bivariate_connectivity.freqs)

### Part 3 - Multivariate connectivity

We will now examine connectivity for some multivariate methods, but before we do so, we need to consider the `indices` parameter again.

#### Indices for bivariate connectivity

`indices` has the form `(seeds, targets)`, where the length of `seeds` and `targets` corresponds to the number of connections.

For bivariate connectivity, `seeds` and `targets` are array-likes of integers, e.g.:
- `seeds=[0, 2, 4]`
- `targets=[1, 3, 5]`

#### Indices for multivariate connectivity

For multivariate connectivity on the other hand, since we are computing connectivity between multiple channels, we need a way to distinguish between the channels belonging to each connection.

Accordingly, we nest the entries for each connection as array-likes within `seeds` and `targets`.

E.g. computing a single multivariate connection between channels 0, 2, and 4 to channels 1, 3, and 5 would require:
- `seeds=[[0, 2, 4]]`
- `targets=[[1, 3, 5]]`.

Note how the length of `seeds` and `targets` still corresponds to the number of connections (in this case, 1).

<br>

E.g. we could compute two multivariate connections with `seeds=[[0], [2, 4]]` and `targets=[[1, 3], [5]]`.

Again, the lengths of `seeds` and `targets` correspond to the number of connections (2), but see how we specify the channels for each connections as a separate array-like.

You may also notice that the number of channels can differ for each connection, making these multivariate methods very flexible.

<br>

More information on the `indices` parameter for multivariate connectivity can be found here: https://mne.tools/mne-connectivity/dev/auto_examples/handling_ragged_arrays.html#sphx-glr-auto-examples-handling-ragged-arrays-py

The image below summarises how indices for bivariate and multivariate methods are handled in MNE-Connectivity:

<img src="figures/connectivity_indices_cheat_sheet.png" alt="Cheat sheet for bivariate and multivariate connectivity indices in MNE-Connectivity" width="60%" height="60%">

#### Multivariate connectivity methods

Earlier we saw that MNE-Connectivity supports multiple bivariate connectivity methods.

Several multivariate methods are also available:
- maximised imaginary part of coherency - [`mic`](https://doi.org/10.1016/j.neuroimage.2011.11.084)
- multivariate interaction measure - [`mim`](https://doi.org/10.1016/j.neuroimage.2011.11.084)
- state-space Granger causality -[`gc`](https://doi.org/10.1103/PhysRevE.91.040101)
- state-space Granger causality on time-reversed signals -[`gc_tr`](https://doi.org/10.1109/TSP.2016.2531628)

Again, references and relevant equations are given in the [documentation](https://mne.tools/mne-connectivity/stable/generated/mne_connectivity.spectral_connectivity_epochs.html#mne_connectivity.spectral_connectivity_epochs).

As for the various bivariate methods, the different multivariate methods enable an appropriate analysis of signals in various contexts.

What is relevant to understand for now is that the maximised imaginary part of coherency (`mic` method) is a multivariate form of the imaginary part of coherency.

**Exercises - multivariate connectivity**

**Exercise:** Compute connectivity using the maximised imaginary part of coherency (`mic` method).

Do this for each interacting pair of channels separately (i.e. 3 connections in total).

In [ ]:
## CODE GOES HERE
multivariate_connectivity_separate = mne_connectivity.spectral_connectivity_epochs(
    data=epochs, method="mic", indices=([[0], [2], [4]], [[1], [3], [5]])
)

**Exercise:** Plot the results for each connection.

How do the pair-wise results for the multivariate method compare to the pair-wise results for the bivariate method above?

*Hint:* We want to take the absolute values of the results.

In [ ]:
## CODE GOES HERE
for con_idx in range(len(multivariate_connectivity_separate.indices[0])):
    plot_connectivity(
        np.abs(multivariate_connectivity_separate.get_data()[con_idx]),
        multivariate_connectivity_separate.freqs,
    )

**Exercise:** Compute connectivity between the same seed and target channels as before but in a single connection.

In [ ]:
## CODE GOES HERE
multivariate_connectivity = mne_connectivity.spectral_connectivity_epochs(
    data=epochs, method="mic", indices=([[0, 2, 4]], [[1, 3, 5]])
)

**Exercise:** Plot the results for this single connection.

How do the results for this single connection of the multivariate method compare to the single connection of the bivariate method which we obtained by averaging?

In [ ]:
## CODE GOES HERE
plot_connectivity(np.abs(multivariate_connectivity.get_data()[0]), multivariate_connectivity.freqs)

### Part 4 - Directed connectivity

So far, the focus has been on coherency-based measures of connectivity.

Coherency-based measures can be very powerful, but they tell us nothing about the direction of the interaction between signals (i.e. they are undirected measures of connectivity).

In contrast, directed measures of connectivity tell us how information is flowing between seeds and targets. Granger causality is one such directed connectivity method.

#### Granger causality

When we created the signals, we simulated the information flow from the seeds to the targets.

As such, we expect Granger causality to be high from `seeds -> targets`, but low from `targets -> seeds`.

**Exercises - Directed connectivity**

**Exercise:** Compute Granger causality (`gc` method) from the seeds to the targets as a single connection and plot the results.

In [ ]:
## CODE GOES HERE
gc_seeds_targets = mne_connectivity.spectral_connectivity_epochs(
    data=epochs, method="gc", indices=([[0, 2, 4]], [[1, 3, 5]])
)
plot_connectivity(gc_seeds_targets.get_data()[0], gc_seeds_targets.freqs)

**Exercise:** Compute Granger causality from the targets to the seeds as a single connection and plot the results.

Are the values of the results lower for `target -> seeds` than `seeds -> targets`?

In [ ]:
## CODE GOES HERE
gc_targets_seeds = mne_connectivity.spectral_connectivity_epochs(
    data=epochs, method="gc", indices=([[1, 3, 5]], [[0, 2, 4]])
)
plot_connectivity(gc_targets_seeds.get_data()[0], gc_targets_seeds.freqs)

**Exercise:** Compute connectivity for both directions (i.e. `seeds -> targets` and `targets -> seeds`) in the same call to `spectral_connectivity_epochs()`.

Plot the results to verify they match those when computed separately.

In [ ]:
## CODE GOES HERE
gc = mne_connectivity.spectral_connectivity_epochs(
    data=epochs, method="gc", indices=([[0, 2, 4], [1, 3, 5]], [[1, 3, 5], [0, 2, 4]])
)

for con_idx in range(2):
    plot_connectivity(gc.get_data()[con_idx], gc.freqs)

#### Investigating bidirectional communication

In neuroscience, we often study systems where information does not only flow in one direction, but reciprocally between brain regions.

Accordingly, examining the **net** directionality of communication can be very useful in identifying the 'drivers' and 'recipients'.

Net Granger causality can be easily computed by subtracting the results of each direction from one another:<br>
`seeds -> targets` - `targets -> seeds`.

**Exercise:** Compute the net Granger scores from the results computed above, and plot the results.

What does this tell us about which set of signals are the 'drivers' and which are the 'recipients'?

In [ ]:
## CODE GOES HERE
plot_connectivity(gc.get_data()[0] - gc.get_data()[1], gc_seeds_targets.freqs)

**Exercise:** Check what happens if we flip the seeds and targets when computing the net Granger scores.

Does this tell us the same thing?

In [ ]:
## CODE GOES HERE
plot_connectivity(gc.get_data()[1] - gc.get_data()[0], gc_seeds_targets.freqs)

As you can see, MNE-Connectivity also supports multivariate methods for investigating connectivity in directed and undirected forms.

## Conclusion

The examples above have been kept simple to demonstrate the basic principles of multivariate connectivity in MNE.

The extensive benefits of multivariate connectivity methods are realised fully in scenarios involving a large number of channels with complex interactions, scenarios where data-driven approaches for extracting the relevant components of connectivity are extremely powerful.

The multivariate methods are also supported by the alternative [`spectral_connectivity_time()`](https://mne.tools/mne-connectivity/stable/generated/mne_connectivity.spectral_connectivity_time.html#mne_connectivity.spectral_connectivity_time) function.

## Additional resources

MNE tutorial on multivariate coherency: https://mne.tools/mne-connectivity/dev/auto_examples/mic_mim.html

MNE tutorial on multivariate Granger causality: https://mne.tools/mne-connectivity/dev/auto_examples/granger_causality.html